### Imports & Model Definition

In [1]:
import gurobipy as gp
from gurobipy import GRB

opt_mod = gp.Model(name="linear program")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-28


#### Data

In [2]:
c = [100, 200, 800, 100, 800, 1200] # cash_required
p = [109, 94.8, 99.5, 93.1, 97.2, 92.9, 110, 104, 102, 95.2] #bond-prices
b_y = [[10, 10, 10, 10, 10, 110],
       [7, 7, 7, 7, 7, 107],
       [8, 8, 8, 8, 8, 108],
       [6, 6, 6, 6, 106, 0],
       [7, 7, 7, 7, 107, 0],
       [5, 5, 5, 105, 0, 0],
       [10, 10, 110, 0, 0, 0],
       [8, 8, 108, 0, 0, 0],
       [7, 107, 0, 0, 0, 0],
       [100, 0, 0, 0, 0, 0]
    ] # ten government bonds with the cash ﬂows and current prices
n = len(p) # 10
m = len(c) # 6


#### Define decision variables

In [3]:
x = opt_mod.addVars(n,
                    name=["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10"],
                    vtype=GRB.CONTINUOUS,
                    lb = 0
)

In [4]:
s = opt_mod.addVars(m,
                    name=["s1","s2","s3","s4","s5","s6"],
                    vtype=GRB.CONTINUOUS,
                    lb = 0
)

#### Add Constraints

In [5]:
for j in range(0, m):
    if j == 0:
        opt_mod.addConstr(sum(x[i]*b_y[i][j] for i in range(0, n)) == c[j] + s[j])
    else:
        opt_mod.addConstr(sum(x[i]*b_y[i][j] for i in range(0, n)) + s[j-1] == c[j] + s[j])


### Set objective Function

In [6]:
obj_fn = sum(p[i]*x[i] for i in range(n))
opt_mod.setObjective(obj_fn, GRB.MINIMIZE)

In [10]:
obj_fn

<gurobi.LinExpr: 109.0 x1 + 94.8 x2 + 99.5 x3 + 93.1 x4 + 97.2 x5 + 92.9 x6 + 110.0 x7 + 104.0 x8 + 102.0 x9 + 95.2 x10>

### Solve the model and output the solution

In [7]:
# opt_mod.setParam("OutputFlag", False)
opt_mod.optimize()
print(f"Optimization is done. Objective Function Value: {opt_mod.ObjVal}")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 16 columns and 52 nonzeros
Model fingerprint: 0x9b4b4833
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [9e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.05s
Presolved: 6 rows, 15 columns, 51 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.125000e+02   0.000000e+00      0s
       4    2.3056916e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.12 seconds (0.00 work units)
Optimal objective  2.305691648e+03
Optimization is done. Objective Function Value: 2305.69164800379


In [8]:
for v in opt_mod.getVars():
    print(f"{v.varName}:{v.x}")

x1:0.0
x2:11.214953271028037
x3:0.0
x4:6.63384512683578
x5:0.0
x6:0.0
x7:0.0
x8:6.008683621172317
x9:0.0
x10:0.0
s1:66.37721262758947
s2:32.75442525517883
s3:0.0
s4:18.30774365821094
s5:0.0
s6:0.0


In [9]:
opt_mod.write("dedication.sol")